In [28]:
import os
import pandas as pd
import numpy as nm
from scipy.sparse import load_npz

#############################
# import necessary packages #
#############################
#import pandas as pd
#import os
#import numpy as np
import seaborn as sns
import matplotlib. pyplot as plt
from sklearn import preprocessing, metrics
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
#from sklearn.datasets import load_breast_cancer
#from dwave.system.samplers import DWaveSampler
#from dwave.system.composites import EmbeddingComposite
#from qboost import QBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from timeit import default_timer as timer
from collections import Counter
from sklearn.metrics import confusion_matrix
from prepare_datasets import scompatta_dataset
from sklearn.metrics import classification_report
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from datetime import datetime

from datetime import datetime
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
import time

In [2]:
def load_dataframe(dir):
    data_filepath = os.path.join(dir, "data.npz")
    index_filepath = os.path.join(dir, "indexes.txt")
    cols_filepath = os.path.join(dir, "cols.txt")
    coo = load_npz(data_filepath)
    with open(index_filepath) as file:
        indexes = file.read().splitlines()
    with open(cols_filepath) as file:
        cols = file.read().splitlines()
    return pd.DataFrame.sparse.from_spmatrix(coo, index=indexes, columns=cols)

In [3]:
def load_features(left_df, dir):
    # Remember that this is made of SparseDType columns
    right_df = load_dataframe(dir)
    right_df.index.name = "cve"
    print(f"Joining Independent Variables")
    full_df = left_df.join(right_df, how="left")
    return full_df

In [4]:
def prepare_dataset_for_ml(dir):
    nvd_filepath = "data/cves/cves.csv"
    exploits_filepath = "data/exploits/cves_exploits.csv"

    nvd_df = pd.read_csv(nvd_filepath, sep=";")
    nvd_df.drop(["pub_date", "description", "references", "cwe"], axis=1, inplace=True)
    nvd_df.set_index("cve", inplace=True)
    #print('Columns :', nvd_df.index)
    nvd_df.rename({col: "NVD_" + col for col in nvd_df.columns}, axis=1, inplace=True)
    #print('data frame :\n', nvd_df)

    #exit()

    exploits_df = pd.read_csv(exploits_filepath)
    #print(exploits_df.columns.tolist())
    exploits_df.drop([col for col in exploits_df.columns.tolist() if col not in ["CVE", "exploitable"]], axis=1, inplace=True)
    #print(exploits_df.columns)
    exploits_df.drop_duplicates(subset=["CVE"], inplace=True)
    exploits_df.set_index("CVE", inplace=True)
    exploits_df.index.name = "cve"
    exploits_df.rename({col: "EDB_" + col for col in exploits_df.columns}, axis=1, inplace=True)
   #print(exploits_df)

    #exit()

    # LEFT JOIN as if no exploit was found it is treated as NOT_EXPLOITED in EDB
    dependent_df = nvd_df.join(exploits_df, how="left")
    dependent_df["EDB_exploitable"] = dependent_df["EDB_exploitable"].fillna(False)

    #print(dependent_df)
    #exit()

    return load_features(dependent_df, dir)

In [5]:
def scompatta_dataset(choosen):
    if choosen == "bow_dir":
        bow_dir = "data/features/bow/"
        if not os.path.exists(os.path.join(bow_dir, "data.npz")):
            print("Missing file with BoW values")
            exit(1)
        bow_df = prepare_dataset_for_ml(bow_dir)
        return bow_df
    
    elif choosen == "tf_dir":
        tf_dir = "data/features/tf/"
        if not os.path.exists(os.path.join(tf_dir, "data.npz")):
            print("Missing file with TF values")
            exit(1)
            tf_df = prepare_dataset_for_ml(tf_dir)
        return tf_df

    elif choosen == "tfidf_dir":
        tfidf_dir = "data/features/tfidf/"
        if not os.path.exists(os.path.join(tfidf_dir, "data.npz")):
            print("Missing file with TF-IDF values")
            exit(1)
        tfidf_df = prepare_dataset_for_ml(tfidf_dir)
        return tfidf_df

In [6]:
if __name__ == "__main__":
    bow_dir = "data/features/bow/"
    tf_dir = "data/features/tf/"
    tfidf_dir = "data/features/tfidf/"
    if not os.path.exists(os.path.join(bow_dir, "data.npz")):
        print("Missing file with BoW values")
        exit(1)
    if not os.path.exists(os.path.join(tf_dir, "data.npz")):
        print("Missing file with TF values")
        exit(1)
    if not os.path.exists(os.path.join(tfidf_dir, "data.npz")):
        print("Missing file with TF-IDF values")
        exit(1)
    
    #bow_df = prepare_dataset_for_ml(bow_dir)
    #tf_df = prepare_dataset_for_ml(tf_dir)
    tfidf_df = prepare_dataset_for_ml(tfidf_dir)

    '''print('Cols :', bow_df.columns)
    print('bow_df :\n', bow_df)

    print('Cols :', tf_df.columns)
    print('tf_df :\n', tf_df)'''

    print('Cols :', tfidf_df.columns)
    print('tfidf_df :\n', tfidf_df)

    '''print('Inizio la creazione dei csv')
    bow_df.to_csv(r'bow_df.csv', index=False, header=True)
    print('finito bow_df')
    tf_df.to_csv(r'tf_df.csv', index=False, header=True)
    print('finito td_df_df')
    tfidf_df.to_csv(r'tfidf_df.csv', index=False, header=True)
    print('finito tfidf_df_df')'''
    print('FINITO')

Joining Independent Variables
Cols : Index(['EDB_exploitable', 'abil', 'abl', 'abnorm', 'abort', 'absolut',
       'abstract', 'abus', 'ac', 'accept',
       ...
       'yahoo', 'ye', 'year', 'yet', 'z', 'zdi', 'zero', 'zip', 'zoho',
       'zone'],
      dtype='object', length=2516)
tfidf_df :
                EDB_exploitable  abil  abl  abnorm  abort  absolut  abstract  \
cve                                                                           
CVE-2019-1347             True   0.0  0.0     0.0    0.0      0.0       0.0   
CVE-2015-8833            False   0.0  0.0     0.0    0.0      0.0       0.0   
CVE-2006-5284             True   0.0  0.0     0.0    0.0      0.0       0.0   
CVE-2020-3795            False   0.0  0.0     0.0    0.0      0.0       0.0   
CVE-2015-4598            False   0.0  0.0     0.0    0.0      0.0       0.0   
...                        ...   ...  ...     ...    ...      ...       ...   
CVE-2015-4291            False   0.0  0.0     0.0    0.0      0.0      

In [7]:
print(tfidf_df['zip'].unique())

[0, 0.31775983535909963, 0.2528842690579323, 0.003587347793290088, 0.02737165572668535, 0.3225755202735132, 0.2777442646525241, 0.11740845103145833, 0.6120393428253902, 0.068846616682496, 0.5481328316641949, 0.09778238017258084, 0.028136451815405698, 0.21132032897660855, 0.5137169867168379, 0.13521294570014247, 0.020054064916887823, 0.23175371088733432, 0.02731915351113213, 0.05964452098618269, 0.303187980942552, 0.27561302862553627, 0.2935365757333482, 0.138489040488428, 0.3363391885474273, 0.17072760275840537, 0.11760601817972088, 0.27574245137014114, 0.05954359661047608, 0.002694367980625139, 0.015041572557670432, 0.2553146068792948, 0.19073509677909806, 0.041734089108035194, 0.3398599088901205, 0.10658554343590075, 0.29533415117721556, 0.13734624628154438, 0.23155596509817433, 0.11221368440200201, 0.06223877808545285, 0.017924578122098526, 0.04549802777995279, 0.31960512932191665, 0.02565605575604424, 0.31999677132246857, 0.059384377248451505, 0.29151248429333443, 0.057646671931376

In [8]:
for i in tfidf_df['abil'].unique():
    print(i, type(i))
    if isinstance(i, int) or isinstance(i, nm.int64):
        print('true')
    else:
        print('false')

0 <class 'int'>
true
0.041960318632391067 <class 'numpy.float64'>
false
0.08602481730282965 <class 'numpy.float64'>
false
0.026179911392083758 <class 'numpy.float64'>
false
0.011051435388987339 <class 'numpy.float64'>
false
0.025226529477374297 <class 'numpy.float64'>
false
0.024186262823393714 <class 'numpy.float64'>
false
0.05702725920361633 <class 'numpy.float64'>
false
0.002981315614238584 <class 'numpy.float64'>
false
0.019984681774546115 <class 'numpy.float64'>
false
0.05117205839604588 <class 'numpy.float64'>
false
0.14506779547379364 <class 'numpy.float64'>
false
0.1406153319655548 <class 'numpy.float64'>
false
0.037456594671375215 <class 'numpy.float64'>
false
0.05261789202969476 <class 'numpy.float64'>
false
0.055962194481133314 <class 'numpy.float64'>
false
0.03160286398122673 <class 'numpy.float64'>
false
0.12540552321858445 <class 'numpy.float64'>
false
0.09042481188648102 <class 'numpy.float64'>
false
0.1431618422095246 <class 'numpy.float64'>
false
0.022130038649657277 <

0.09652337105847074 <class 'numpy.float64'>
false
0.017755197825287096 <class 'numpy.float64'>
false
0.04482591678008934 <class 'numpy.float64'>
false
0.016105889195102224 <class 'numpy.float64'>
false
0.05647543818217696 <class 'numpy.float64'>
false
0.06692548750002689 <class 'numpy.float64'>
false
0.20064074756211403 <class 'numpy.float64'>
false
0.07559405773296343 <class 'numpy.float64'>
false
0.08748155782579241 <class 'numpy.float64'>
false
0.07593513736143381 <class 'numpy.float64'>
false
0.0626813479967415 <class 'numpy.float64'>
false
0.01604637228464907 <class 'numpy.float64'>
false
0.30148726949715426 <class 'numpy.float64'>
false
0.08101596163023358 <class 'numpy.float64'>
false
0.06349200089559058 <class 'numpy.float64'>
false
0.031334850600259985 <class 'numpy.float64'>
false
0.08859439426607281 <class 'numpy.float64'>
false
0.19414988045954118 <class 'numpy.float64'>
false
0.027548740433846128 <class 'numpy.float64'>
false
0.06411579417172657 <class 'numpy.float64'>
fal

0.05295532311448356 <class 'numpy.float64'>
false
0.044792229937304584 <class 'numpy.float64'>
false
0.24321673884853792 <class 'numpy.float64'>
false
0.20583083395650464 <class 'numpy.float64'>
false
0.17170730563881242 <class 'numpy.float64'>
false
0.02645230174011164 <class 'numpy.float64'>
false
0.0677977206617934 <class 'numpy.float64'>
false
0.04985453004350419 <class 'numpy.float64'>
false
0.067917393032132 <class 'numpy.float64'>
false
0.07969821234670861 <class 'numpy.float64'>
false
0.039481240322062246 <class 'numpy.float64'>
false
0.09386548100374495 <class 'numpy.float64'>
false
0.19510097077856853 <class 'numpy.float64'>
false
0.06826287299949817 <class 'numpy.float64'>
false
0.040319955836571754 <class 'numpy.float64'>
false
0.08184663928277519 <class 'numpy.float64'>
false
0.1625594248526541 <class 'numpy.float64'>
false
0.16328036378585342 <class 'numpy.float64'>
false
0.174975023518079 <class 'numpy.float64'>
false
0.14205024449370107 <class 'numpy.float64'>
false
0.0

In [7]:
dataset = "tfidf_dir"
percentage = 0.20

In [8]:
def convert_labels(y_data):
    """
    :param y_data: labels 
    :return: array of numerical labels (0:non-malicious,1:malicious)
    """
    y_data = y_data.values.ravel()
    y_data = np.where(y_data, 1, -1)

    return y_data

In [16]:
def remove_multiclass(df):
    df = df[df['label'].isin([0, 1])]
    df = df.sample(frac=percentage)
    return df

In [18]:
################
# Load dataset #
################
#choosen = scompatta_dataset(dataset)
#print('Dataset loaded')

#Dataset already loaded in this case

In [25]:
# Remove Multiclass 
choosen = remove_multiclass(tfidf_df)

In [40]:
choosen.shape[0]

29529

In [41]:
tfidf_df.shape[0]

147936

In [37]:
tfidf_df['label'].isin([0, 1])

cve
CVE-2019-1347    True
CVE-2015-8833    True
CVE-2006-5284    True
CVE-2020-3795    True
CVE-2015-4598    True
                 ... 
CVE-2015-4291    True
CVE-2006-5337    True
CVE-2012-0870    True
CVE-2019-6794    True
CVE-2008-6394    True
Name: label, Length: 147936, dtype: bool

In [53]:
label_val=tfidf_df['label'].unique()

In [42]:
cols = list(choosen.columns)

In [56]:
print(label_val.min())

0


In [67]:
tfidf_df["EDB_exploitable"]

cve
CVE-2019-1347     True
CVE-2015-8833    False
CVE-2006-5284     True
CVE-2020-3795    False
CVE-2015-4598    False
                 ...  
CVE-2015-4291    False
CVE-2006-5337    False
CVE-2012-0870    False
CVE-2019-6794    False
CVE-2008-6394     True
Name: EDB_exploitable, Length: 147936, dtype: bool

In [9]:
def convert_labels(y_data):
    """
    :param y_data: labels 
    :return: array of numerical labels (0:non-malicious,1:malicious)
    """
    y_data = y_data.values.ravel()
    y_data = np.where(y_data, 1, -1)

    return y_data

In [10]:
percentage = 0.20 #percentage of dataset

In [11]:
def transform_dependent_values(y_data):
    y_data = y_data.values.ravel()
    y_data = np.where(y_data, 1, -1)

    return y_data

In [12]:
choosen=tfidf_df

In [13]:
#######################################
# Removing label from dataset columns #
#######################################
print('splitting dataset... ')
cols = list(choosen.columns)
# cols.remove('label')
cols.remove('EDB_exploitable')
choosen['EDB_exploitable'] = choosen['EDB_exploitable'].astype(int)

splitting dataset... 


In [14]:
choosen

,EDB_exploitable,abil,abl,abnorm,abort,absolut,abstract,abus,ac,accept,...,yahoo,ye,year,yet,z,zdi,zero,zip,zoho,zone
cve,,,,,,,,,,,,,,,,,,,,,
CVE-2019-1347,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2015-8833,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2006-5284,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2020-3795,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2015-4598,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CVE-2015-4291,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2006-5337,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CVE-2012-0870,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#####################################
# Split columns for indipendentList #
#####################################
x_train,x_test,y_train,y_test=train_test_split(choosen[cols],choosen[['EDB_exploitable']],test_size=0.2,shuffle=True)

In [20]:
y_train

,EDB_exploitable
cve,
CVE-2008-5962,1
CVE-2004-0654,0
CVE-2013-5771,0
CVE-2019-1297,0
CVE-2018-18705,0
...,...
CVE-2019-20079,0
CVE-2018-12047,0
CVE-2016-4145,0


In [21]:
############################
# Convert labels into 1/-1 #
############################
y_train = convert_labels(y_train)
y_test = convert_labels(y_test)

In [25]:
y_train

array([ 1, -1, -1, ..., -1, -1, -1])

In [26]:
################################################
# Create svc model and fit it on training data #
################################################
'''print('training svc model... ')
svc = SVC(kernel='linear', C = 1.0)
start = timer()
svc.fit(x_train, y_train)
end = timer()
print(f'SVC training time in seconds: {(end - start)}')
tempo = (end - start)'''

"print('training svc model... ')\nsvc = SVC(kernel='linear', C = 1.0)\nstart = timer()\nsvc.fit(x_train, y_train)\nend = timer()\nprint(f'SVC training time in seconds: {(end - start)}')\ntempo = (end - start)"

In [ ]:
start = time.time()
# Data Training
# Normal / SVC
print("Training Normal/SVC ...")
clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(x_train, y_train)
end = time.time()

tempo_normal = end - start
print(tempo_normal)

Training Normal/SVC ...


In [99]:
# ------------TRAIN CON ADABOOST DECOMMENTARE E COMMENTARE SVC
#clf = AdaBoostClassifier(base_estimator=None, n_estimators=35, algorithm='SAMME.R', random_state=None)
#clf.fit(x_train, y_train)

AdaBoostClassifier(n_estimators=35)

In [ ]:
print("hello world")